# 🧠 LLM Agent (MVP) Using a Local Model

This beginner notebook demonstrates a basic agent architecture using a **local LLM** like one run via [Ollama](https://ollama.com), LM Studio, or similar interfaces.

### ✅ Features:
- Simple agent loop
- Tool use: calculator
- Local model integration via HTTP (OpenAI-compatible API or other)

👉 Make sure you have a local LLM running (like `llama3`, `mistral`, etc.) via Ollama or LM Studio.

In [ ]:
# 📦 Install required packages
# !pip install requests

In [ ]:
import requests
import json

## 🧮 Calculator Tool

In [ ]:
def calculator(expression: str) -> str:
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Calculator Error: {str(e)}"

## 🔍 Decide if we should use a tool

In [ ]:
def decide_tool(prompt: str) -> str:
    if any(op in prompt for op in ["+", "-", "*", "/"]):
        return "calculator"
    return "none"

## 🤖 Function to query your local LLM API
- This example assumes you're using **Ollama** (running at `http://localhost:11434`) with a model like `llama3`
- You can adjust the URL and payload format depending on your local setup

In [ ]:
def query_local_llm(prompt: str, model: str = "llama3") -> str:
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        return data.get("response", "[No response from LLM]")
    except Exception as e:
        return f"[Error calling local LLM: {str(e)}]"

## 🔁 Run the Agent

In [ ]:
def run_local_agent(user_input: str):
    tool = decide_tool(user_input)
    thoughts = [f"User asked: {user_input}"]

    if tool == "calculator":
        result = calculator(user_input)
        thoughts.append(f"Used calculator: {result}")
        final_answer = result
    else:
        result = query_local_llm(user_input)
        thoughts.append("Used local LLM.")
        final_answer = result

    print("\n--- Thoughts ---")
    for t in thoughts:
        print("•", t)
    print("\n--- Answer ---")
    print(final_answer)

## 🚀 Try the Agent!

In [ ]:
while True:
    user_input = input("\n🧑 What do you want to do? (type 'exit' to quit)\n> ")
    if user_input.lower() == "exit":
        break
    run_local_agent(user_input)